In [3]:
import fitz

In [4]:
src = fitz.open("../PyMuPDF/570_augustin_fertig_kontrolle.pdf")
new_doc = fitz.open()  # empty output PDF

page = src.load_page(1)
r = page.rect
d = fitz.Rect(
    # CropBox displacement if being set
    page.cropbox_position,
    page.cropbox_position,
)  # starting at (0, 0)

r1 = r
r1.x1 = r1.x1 / 2  # left side of double page
r2 = r1 + (r1.width, 0, r1.width, 0)  # right side of double page
# r2 = r + (r.width / 2, 0, 0, 0)  # right side of double page

r2 += d  # add the CropBox displacement
new_page = new_doc.new_page(-1, width=r2.width, height=r2.height)
new_page.show_pdf_page(
    new_page.rect,
    src,
    page.number,
    clip=r2,
)

107

In [5]:
new_page.get_text("text")

'570\n3\naugustiner:in\nBernd Pegritz \n«Rätsel» lösen mit Bildern\nPROTOKOLL: SÓNIA MELO \nFOTO: MARIO LANG\nA\nnstatt die Hausaufgaben zu machen, \nhabe ich als Kind mal das Logo von \nMozilla Firefox nachgezeichnet. \n Irgendwann habe ich aufgeschnappt, \ndass es Leute gibt, die dafür bezahlt werden. Die \nEntscheidung Grafikdesigner zu werden, fiel \nfrüh, lange bevor ich wusste, was das genau ist.\nIn Tirol geboren und aufgewachsen, zog ich \nfür das Masterstudium in Kommunikations-\ndesign nach Saarbrücken in Deutschland. Im \nStudium habe ich immer mehr  Illustration ein-\ngebaut. Nun geht die Entwicklung weiter mit \nAnimation, weil ich es spannend finde, wenn \ndie Zeichnungen zum Leben erwachen und \nman in einer ganz anderen Erzähl-Struktur \ndenken muss.\nIch bin wie ein «Rätsellöser», dabei sind \nmeine Antworten keine Wörter, sondern Bilder. \nDiese Aufgabe, einen Inhalt auf den Punkt zu \nbringen und in einem oder in wenigen Bildern \neine zusätzliche Ebene zu schaffen, 